# Runnables

## Métodos dos Runnables de LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- **stream:** transmitir de volta fragmentos da resposta
- **invoke:** chamar a cadeia com um input
- **batch:** chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência:

- **astream:** transmitir de volta fragmentos da resposta de forma assíncrona
- **ainvoke:** chamar a cadeia com um input de forma assíncrona
- **abatch:** chamar a cadeia com uma lista de inputs de forma assíncrona

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")

chain = prompt | model

## Invoke

O invoke é o método básico para inserir uma input na cadeia e receber uma resposta

In [2]:
chain.invoke({'assunto': 'cachorrinhos'})

AIMessage(content='Os cachorrinhos são companheiros leais e amorosos que iluminam nossos dias com sua alegria e carinho.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 20, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c3971a54-cfee-4545-8765-03fbe1f5bfe0-0', usage_metadata={'input_tokens': 20, 'output_tokens': 31, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Ele pode ser rodado como uma simples string quando existe apenas uma input no prompt, mas a forma mais recomendada é informando especificamente o nome da input através de um dicionário.

In [3]:
chain.invoke('cachorrinhos')

AIMessage(content='Cachorrinhos são a prova de que o amor incondicional pode vir em um pacote peludo e cheio de lambidas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 20, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-277281c3-fc57-41d9-bfe7-06a215d72a9b-0', usage_metadata={'input_tokens': 20, 'output_tokens': 31, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Stream

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream


In [4]:
for stream in chain.stream('cachorrinhos'):
    print(stream.content, end='') 

Cachorrinhos são como pequenos raios de sol que iluminam nossos dias com amor e alegria.

## Batch

Para fazermos múltimpas requisições em paralelo utilizamos o batch

In [5]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinhos'}, {'assunto': 'cavalinhos'}])

[AIMessage(content='Cachorrinhos são como pequenos raios de sol que iluminam nossas vidas com amor e alegria.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 20, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e8992de0-cba7-42ae-9cc0-6a1748123f31-0', usage_metadata={'input_tokens': 20, 'output_tokens': 30, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='"Os gatinhos encantam com seus olhares curiosos e seus ronrons carinhosos, trazendo alegria e conforto para nossas vidas."', additional_kwargs={'refusal': None}, response_metadata=

In [6]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinhos'}, {'assunto': 'cavalinhos'}], config={'max_concurrency': 2})

[AIMessage(content='Cachorrinhos são como pequenos anjos de quatro patas que trazem alegria e amor para nossas vidas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 20, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e8c01b27-c31a-424a-9ce3-be62c11d8b0c-0', usage_metadata={'input_tokens': 20, 'output_tokens': 32, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='"Os gatinhos são como pequenos feitiços que encantam nossos corações com suas travessuras e carinhos."', additional_kwargs={'refusal': None}, response_metadata={'token_usag

## Runnables especiais

### Rodando em paralelo
```
     Input      
      / \       
     /   \      
 Branch1 Branch2
     \   /      
      \ /       
      Combine   
```

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Crie um nome para o seguinte produto: {produto}")

chain_nome = prompt | model | StrOutputParser()

In [8]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Descreva o cliente potencial para o seguinte produto: {produto}")

chain_clientes = prompt | model | StrOutputParser()

In [9]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("""Dado o produto com o seguinte nome e seguinte
público potencial, desenvolva um anúncio para o produto.
                                          
Nome do produto: {nome_produto}
Público: {publico}""")

In [10]:
parallel = RunnableParallel({'nome_produto': chain_nome, 'publico': chain_clientes})
parallel.invoke({'produto': 'Um copo inquebrável'})

{'nome_produto': 'SuperCopo Invencível',
 'publico': 'O cliente potencial para um copo inquebrável pode ser alguém que tem crianças em casa e está preocupado com a segurança delas, pois evita acidentes com vidro quebrado. Além disso, pessoas que gostam de praticar esportes ao ar livre, como camping, trilhas e atividades radicais, podem se interessar por um copo inquebrável por sua durabilidade e resistência. Outro público-alvo pode ser pessoas que são desastradas e costumam derrubar objetos com frequência, pois um copo inquebrável seria uma opção mais prática e econômica a longo prazo.'}

In [11]:
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Um copo inquebrável'})

'Anúncio:\n\nChega de se preocupar com copos quebrados! Apresentamos o DurávelGlass, o copo inquebrável perfeito para sua casa, restaurante, festa ou aventura ao ar livre.\n\nFeito com material resistente e durável, o DurávelGlass é a opção ideal para quem busca praticidade e segurança. Com ele, você não precisa mais se preocupar com quedas, batidas ou crianças agitadas em casa.\n\nSeja para servir bebidas em uma festa animada, para levar em uma trilha ou acampamento, ou simplesmente para uso diário em casa, o DurávelGlass é o companheiro perfeito para todas as ocasiões.\n\nNão perca mais tempo quebrando copos frágeis e invista em durabilidade com o DurávelGlass. Garanta já o seu e tenha a tranquilidade de um copo que não quebra!'

In [12]:
from langchain_core.runnables import RunnableLambda

def cumprimentar(nome):
    return f'Olá, {nome}!'

runnable_cumprimentar = RunnableLambda(cumprimentar)

resultado = runnable_cumprimentar.invoke('Maria')
print(resultado)

Olá, Maria!


In [13]:
from langchain_core.runnables import RunnablePassthrough
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("""Dado o produto com o seguinte nome e seguinte
público potencial, desenvolva um anúncio para o produto.
                                          
Nome do produto: {nome_produto}
Característica do produto: {produto}
Público: {publico}""")

parallel = RunnablePassthrough().assign(**{'nome_produto': chain_nome, 'publico': chain_clientes})
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Copo inquebrável'})